# PYTORCH TENSORS - ADVANCED

In [1]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 20px;
}

div.output_area pre {
    font-size: 20px;
}

div.text_cell_render {
    font-size: 20px;
}
</style>

In [2]:
%matplotlib inline

In [3]:
import torch
import warnings
warnings.filterwarnings('ignore')

## torch.tensor vs. torch.Tensor

In [7]:
i = torch.tensor(1)  # gives you a tensor with fixed value of 1
j = torch.Tensor(1)  # gives you a tensor of size 1 which is randomly initialized
i, j

(tensor(1), tensor([0.]))

## Tensor Memory Size

In [8]:
import sys
a = 45   # int64
b = 45.2 # float64
print(type(a))
print(type(b))
print(sys.getsizeof(a)) # prints 28 bytes
print(sys.getsizeof(b)) # prints 24 bytes

# https://stackoverflow.com/questions/31469553/python-memory-consumption-in-64-bit-system-for-int-and-float
# 64 bit system
# int:   overhead = 20 bytes, value = 8 bytes
# float: overhead = 16 bytes, value = 8 bytes

<class 'int'>
<class 'float'>
28
24


In [9]:
import numpy as np
n_int = np.array([1, 2, 3])         # int64
n_float = np.array([1.0, 2.0, 3.0, 4.0]) # float64
print(n_int.dtype)
print(n_float.dtype)
print()

print(n_int.size)
print(n_float.size)
print()

print(n_int.itemsize)
print(n_float.itemsize)
print()

print(n_int.size * n_int.itemsize)
print(n_float.size * n_float.itemsize)
print()

print(n_int.nbytes)
print(n_float.nbytes)
print()

print(sys.getsizeof(n_int)) # prints 136 bytes
print(sys.getsizeof(n_float)) # prints 144 bytes

int64
float64

3
4

8
8

24
32

24
32

136
144


In [10]:
import torch
t_int = torch.tensor([1, 2, 3])                                # int64
t_float_1 = torch.tensor([1.0, 2.0, 3.0])                       # flaot32 !!!!!!
t_float_2 = torch.tensor([1.0, 2.0, 3.0, 4.0])                  # flaot32 !!!!!!
t_float_3 = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float64)   # flaot64 !!!!!!
print(t_int.dtype)
print(t_float_1.dtype)
print(t_float_2.dtype)
print(t_float_3.dtype)
print()

print(t_int.element_size())
print(t_float_1.element_size())
print(t_float_2.element_size())
print(t_float_3.element_size())
print()

print(t_int.nelement())
print(t_float_1.nelement())
print(t_float_2.nelement())
print(t_float_3.nelement())
print()

print(t_int.element_size() * t_int.nelement())
print(t_float_1.element_size() * t_float_1.nelement())
print(t_float_2.element_size() * t_float_2.nelement())
print(t_float_3.element_size() * t_float_3.nelement())
print()

print(sys.getsizeof(t_int)) # prints 72 bytes
print(sys.getsizeof(t_float_1)) # prints 72 bytes
print(sys.getsizeof(t_float_2)) # prints 72 bytes
print(sys.getsizeof(t_float_3)) # prints 72 bytes

torch.int64
torch.float32
torch.float32
torch.float64

8
4
4
8

3
3
4
3

24
12
16
24

72
72
72
72


- the torch.Tensor object merely holds a reference to the actual memory (storage)

## Memory: List vs. Tensor

![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure2.png)

In [11]:
a0 = 1.0
a1 = 2.0
a2 = 3.0

print(id(a0) , id(a1), id(a2))

5944181360 5941000080 5940998544


In [12]:
a = [a0, a1, a2]

print(id(a[0]) , id(a[1]), id(a[2]))
print(id(a))

5944181360 5941000080 5940998544
5944266560


In [13]:
import torch
c = torch.tensor([a0, a1, a2])

print(c[0].data_ptr(), c[1].data_ptr(), c[2].data_ptr())
print(c.data_ptr())
print(c[0], c[1], c[2])
print(c)

4369949056 4369949060 4369949064
4369949056
tensor(1.) tensor(2.) tensor(3.)
tensor([1., 2., 3.])


## Efficient Memory Management

In [14]:
data = [[1.0, 2.0],
        [3.0, 4.0],
        [5.0, 6.0]]
a = torch.tensor(data)
print(a)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


In [15]:
b = a[1,:]
print(b)

tensor([3., 4.])


In [16]:
b[0] = -999
print(b)
print(a)

tensor([-999.,    4.])
tensor([[   1.,    2.],
        [-999.,    4.],
        [   5.,    6.]])


## Storage, Offset, Stride
- https://hiddenbeginner.github.io/deeplearning/2020/01/21/pytorch_tensor.html
- 1차원 텐서이든, 2차원 텐서이든 혹은 d차원 텐서이든 모든 텐서는 메모리에 1차원 배열의 형태로 저장됨
- storage: 텐서를 효율적으로 다룰 수 있도록 만들어주는 이러한 1차원 배열 (인덱싱한 텐서의 값에 접근할 때 offset과 stride 사용)
![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure3.png)
  - 위 두 텐서에 대하여 storage의 offset은 모두 0
  - stride는 텐서 안의 어떤 한 값에서 다음 원소를 얻기 위하여 storage에서 뛰어 넘어야할 인덱스의 개수
    - storage의 stride는 텐서의 차원만큼 정의됨

In [17]:
a = torch.tensor(
    [[3,1,8],
     [0,9,2]]
)
print("============storage==========")
print(a.storage())
print("=============================")
print("Offset: ", a.storage_offset())
print("Strides: ", a.stride())

============storage==========
 3
 1
 8
 0
 9
 2
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
Offset:  0
Strides:  (3, 1)


In [18]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

### tenset[i, j] == storage[storage_offset + i * stirde[0] + j * stirde[1]]
### tenset[i, j, k] == storage[storage_offset + i * stirde[0] + j * stirde[1] + k * stirde[2]]

In [19]:
x1 = torch.arange(12).view((4, 3))
x1

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [20]:
x1.stride()

(3, 1)

In [21]:
x2 = x1.view(3, 4)
x2

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [22]:
x2.stride()

(4, 1)

In [23]:
storage = x2.storage()
storage

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

In [24]:
offset = x2.storage_offset()
stride = x2.stride()
offset, stride

(0, (4, 1))

In [25]:
item1 = x2[2, 1].item()
item1

9

In [26]:
item2 = storage[offset + 2 * x2.stride()[0] + 1 * x2.stride()[1]] # storage[0 + 2 * 4 + 1 * 1] = storage[9]
item2

9

In [27]:
item3 = storage[9]
item3

9

In [28]:
x = torch.arange(1200).view((10, 30, 4))
storage = x.storage()
offset = x.storage_offset()
stride = x.stride()
offset, stride

(0, (120, 4, 1))

In [29]:
item1 = x[4, 23, 2].item()
item2 = storage[offset + 4 * stride[0] + 23 * stride[1] + 2 * stride[2]]
item1 == item2

True

![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure4.png)

In [30]:
a = torch.tensor(
    [[3,1,1,2],
     [8,0,3,4],
     [9,2,5,6]]
)
print("============storage==========")
print(a.storage())
print(id(a.storage()))
print("=============================")
print("Offset: ", a.storage_offset())
print("Strides: ", a.stride())

============storage==========
 3
 1
 1
 2
 8
 0
 3
 4
 9
 2
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]
4382079248
Offset:  0
Strides:  (4, 1)


In [31]:
b = a[1:3, 1:3]
print("============storage==========")
print(b.storage())
print(id(b.storage()))
print("=============================")
print("Offset: ", b.storage_offset())
print("Strides: ", b.stride())
storage = b.storage()
offset = b.storage_offset()
stride = b.stride()

============storage==========
 3
 1
 1
 2
 8
 0
 3
 4
 9
 2
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]
4382080688
Offset:  5
Strides:  (4, 1)


In [32]:
id(a.storage()) == id(b.storage())

False

In [33]:
id(a.untyped_storage()) == id(b.untyped_storage())

True

In [34]:
b

tensor([[0, 3],
        [2, 5]])

In [35]:
item1 = b[1, 1].item()
item1

5

In [36]:
item2 = storage[offset + 1 * stride[0] + 1 * stride[1]]
item1 == item2

True

In [37]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
points_storage = points.storage()
points_storage[0]

4.0

In [38]:
points.storage()[1]

1.0

In [39]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [40]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
points.stride()

(2, 1)

In [41]:
second_point = points[1]
second_point

tensor([5., 3.])

In [42]:
second_point.storage_offset()

2

In [43]:
second_point.size()

torch.Size([2])

In [44]:
second_point.shape

torch.Size([2])

In [45]:
second_point.stride()

(1,)

In [46]:
item1 = points[1, 1].item()
item1

3.0

In [47]:
item2 = second_point[1].item()
item2

3.0

In [48]:
item3 = points_storage[second_point.storage_offset() + 1 * second_point.stride()[0]]
item3

3.0

In [49]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [50]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [51]:
points = torch.tensor(
    [[4.0, 1.0],
     [5.0, 3.0],
     [2.0, 1.0]]
)
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [52]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [53]:
id(points.storage()) == id(points_t.storage())

False

In [54]:
id(points.untyped_storage()) == id(points_t.untyped_storage())

True

In [55]:
points.stride()

(2, 1)

In [56]:
points_t.stride()

(1, 2)

In [57]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [58]:
points.is_contiguous()

True

In [59]:
for i in range(3):
    for j in range(2):
        print(points[i][j].data_ptr())

4924523008
4924523012
4924523016
4924523020
4924523024
4924523028


In [60]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [61]:
points_t.is_contiguous()

False

In [62]:
for i in range(2):
    for j in range(3):
        print(points_t[i][j].data_ptr())

4924523008
4924523016
4924523024
4924523012
4924523020
4924523028


In [87]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [88]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [89]:
points_t.stride()

(1, 2)

In [90]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [91]:
points_t_cont.stride()

(3, 1)

In [92]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

- https://stackoverflow.com/questions/48915810/what-does-contiguous-do-in-pytorch

## view vs. transpose vs. reshape
- https://inmoonlight.github.io/2021/03/03/PyTorch-view-transpose-reshape/
- contiguos
  - A tensor is considered contiguous if its memory layout is such that the elements are stored in adjacent memory addresses.
  - Some operations require that the tensor be contiguous in order to work properly.

In [9]:
t = torch.tensor([[[0, 1],
                   [2, 3],
                   [4, 5]],
                  [[6, 7],
                   [8, 9],
                   [10, 11]],
                  [[12, 13],
                   [14, 15],
                   [16, 17]],
                  [[18, 19],
                   [20, 21],
                   [22, 23]]])  # (4, 3, 2)
t.shape

torch.Size([4, 3, 2])

In [10]:
tv = t.view(4, 2, 3)
tv

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23]]])

In [11]:
tv.shape

torch.Size([4, 2, 3])

In [12]:
tv.is_contiguous()

True

In [13]:
t = torch.tensor([[[0, 1],
                   [2, 3],
                   [4, 5]],
                  [[6, 7],
                   [8, 9],
                   [10, 11]],
                  [[12, 13],
                   [14, 15],
                   [16, 17]],
                  [[18, 19],
                   [20, 21],
                   [22, 23]]])  # (4, 3, 2)
t.shape

torch.Size([4, 3, 2])

#### transpose
- tensor 에 저장된 데이터의 물리적 위치 순서와 상관없이 수학적 의미의 transpose를 수행한다.
- 따라서, 일반적으로 transpose가 수행된 tensor는 항상 contiguous 하지 않다.

In [14]:
tr = t.transpose(2, 1)
tr.shape

torch.Size([4, 2, 3])

In [15]:
tr

tensor([[[ 0,  2,  4],
         [ 1,  3,  5]],

        [[ 6,  8, 10],
         [ 7,  9, 11]],

        [[12, 14, 16],
         [13, 15, 17]],

        [[18, 20, 22],
         [19, 21, 23]]])

In [16]:
tr.is_contiguous()

False

#### view 는 contiguous 하지 않은 tensor에 대해서는 적용할 수 없다.

In [17]:
tr.view(4, 3, 2)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [18]:
tr2 = tr.contiguous()
tr2

tensor([[[ 0,  2,  4],
         [ 1,  3,  5]],

        [[ 6,  8, 10],
         [ 7,  9, 11]],

        [[12, 14, 16],
         [13, 15, 17]],

        [[18, 20, 22],
         [19, 21, 23]]])

In [19]:
tr3 = tr2.view(4, 3, 2)
tr3

tensor([[[ 0,  2],
         [ 4,  1],
         [ 3,  5]],

        [[ 6,  8],
         [10,  7],
         [ 9, 11]],

        [[12, 14],
         [16, 13],
         [15, 17]],

        [[18, 20],
         [22, 19],
         [21, 23]]])

#### tensor.reshape() == tensor.contiguous().view()
- reshape 은 강제로 tensor를 contiguous 하게 만들면서 shape을 변경

In [20]:
tr4 = tr.view(4, 3, 2)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [21]:
tr4 = tr.reshape(4, 3, 2)
tr4

tensor([[[ 0,  2],
         [ 4,  1],
         [ 3,  5]],

        [[ 6,  8],
         [10,  7],
         [ 9, 11]],

        [[12, 14],
         [16, 13],
         [15, 17]],

        [[18, 20],
         [22, 19],
         [21, 23]]])

## Working with NumPy

In [108]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [109]:
id(points) == id(points_np)

False

- points & points_np are objects of different Python types, so, as far as Python is concerned, they have different addresses
- However, they share an underlying C array in memory & modifying one in-place also modies the other

In [110]:
points.zero_()
points

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [111]:
print(points_np)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [112]:
points2 = torch.from_numpy(points_np)

In [113]:
points2.add_(10)
points2

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.],
        [10., 10., 10., 10.]])

In [114]:
points_np

array([[10., 10., 10., 10.],
       [10., 10., 10., 10.],
       [10., 10., 10., 10.]], dtype=float32)

In [115]:
points

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.],
        [10., 10., 10., 10.]])

## torch save & load

In [116]:
torch.save(points, '../data/p1ch3/ourpoints.t')

In [117]:
with open('../data/p1ch3/ourpoints.t','wb') as f:
   torch.save(points, f)

In [118]:
points = torch.load('../data/p1ch3/ourpoints.t')
points

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.],
        [10., 10., 10., 10.]])

In [119]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [120]:
with open('../data/p1ch3/ourpoints.t','rb') as f:
   points = torch.load(f)

In [121]:
#  pip install h5py
import h5py

f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()
dset

<Closed HDF5 dataset>

In [122]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
dset

<HDF5 dataset "coords": shape (3, 4), type "<f4">

In [123]:
last_points = torch.from_numpy(dset[1:])
f.close()

In [124]:
last_points

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.]])

In [125]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points_gpu = points.to(device='cuda')

In [126]:
points_gpu = points.to(device='cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [127]:
points = 2 * points  # <1>
points_gpu = 2 * points.to(device='cuda')  # <2>

AssertionError: Torch not compiled with CUDA enabled

In [128]:
points_gpu = points_gpu + 4

NameError: name 'points_gpu' is not defined

In [129]:
points_cpu = points_gpu.to(device='cpu')

NameError: name 'points_gpu' is not defined

In [130]:
points_gpu = points.cuda()  # <1>
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

AssertionError: Torch not compiled with CUDA enabled

In [131]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [132]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [133]:
a = torch.ones(3, 2)

In [134]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## torch.dot vs. torch.matmul vs. torch.mm vs. torch.bmm
- https://velog.io/@regista/torch.dot-torch.matmul-torch.mm-torch.bmm

In [135]:
batch1 = torch.randn(10, 3, 20, 10)
batch2 = torch.randn(10, 3, 10, 30)
res = torch.matmul(batch1, batch2)
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 3, 20, 30])

In [136]:
batch1 = torch.randn(10, 3, 20, 10)
batch2 = torch.randn(10, 3, 10, 30)
res = batch1 @ batch2
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 3, 20, 30])

In [137]:
batch1 = torch.randn(10, 20, 10)
batch2 = torch.randn(10, 10, 30)
res = torch.bmm(batch1, batch2)
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 20, 30])

- https://pytorch.org/docs/stable/generated/torch.bmm.html
- mat1 and mat2 must be 3-D tensors